In [4]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 0C52-665E

 Directory of C:\Users\lic\Documents\PythonScripts\BostonMarathon

03/02/2017  01:53 PM    <DIR>          .
03/02/2017  01:53 PM    <DIR>          ..
03/02/2017  10:07 AM    <DIR>          .ipynb_checkpoints
03/02/2017  11:37 AM             8,098 BMwithsplits.ipynb
03/02/2017  10:06 AM            33,017 BostonMarathon.ipynb
03/02/2017  09:59 AM    <DIR>          Data
03/02/2017  09:59 AM                63 README.md
               3 File(s)         41,178 bytes
               4 Dir(s)  28,721,311,744 bytes free


In [13]:
import requests
from bs4 import BeautifulSoup
import os.path
import pandas as pd
import re
import string
BAAsite = 'http://registration.baa.org/'
url_suffix = '/cf/Public/iframe_ResultsSearch.cfm?mode=results'
columns=['bib','name','age','gender','city','state', 'country','ctz',
         't5k','t10k','t15k', 't20k','half','t25k','t30k','t35k','t40k',
         'pace','projt','offt','overcall','gdiv','div']

#2014 ma
#gender 0, 1: M, 2: F,

In [18]:
def request_with_year_lastname_gender(year,lastname,gender=0,start=1):
    param = {     'StoredProcParamsOn': 'yes',
                 'LastName': lastname,
                 'GenderID': gender,
                 'VarTargetCount': 1000,
                 'records':25,
                 'start': start,
                 "RaceYearLowID": str(year),
                 'next': 'Next+25+Records'}
    url=BAAsite+str(year)+url_suffix
    r = requests.post(url, param)
    return r

def save_splitdata_tocsv(df_bm,year):
    filename=''.join(['bm_split',str(year),'.csv'])
    if os.path.isfile(filename):
        with open(filename,'a') as f:
            df_bm.to_csv(f,index = False,header=False,encoding='utf-8')
    else:
        df_bm.to_csv(filename,index = False,header=True,encoding='utf-8')
        
def bm_split_list(request_result):
    df_bm=pd.DataFrame(columns=columns)
    soup = BeautifulSoup(request_result.text,"lxml")
    maintable=soup.find_all(name='tr', attrs={'class': 'tr_header'})
    subtable = soup.find_all(name="table", attrs={"class": "table_infogrid"})
    num=len(maintable)
    if num>0:
        for i in range(num):
            (bib,name,age,gender,city,state,country,ctz,_)=[t.text.strip() for t in maintable[i].findAll("td")]
            (t5k, t10k, t15k, t20k,half,t25k,t30k,t35k,t40k,pace,projt,offt,overcall,gdiv,div )= [t.text.strip() for t in subtable[i].findAll("td")]
            df_bm.loc[i]=[bib,name,age,gender,city,state,country,ctz,t5k, t10k, t15k, t20k,half,t25k,t30k,t35k,t40k,pace,projt,offt,overcall,gdiv,div]
    return df_bm, num

In [4]:
string.ascii_lowercase  #python 3? 

'abcdefghijklmnopqrstuvwxyz'

In [30]:
def get_list_year_lastname(year=2014, lastname='ma'):
    for gender in [1,2]:
        print('lastname:{},gender:{}'.format(lastname,gender),end='')
        totalnum=0
        for start in list(range(1,1000,25)):
            print('{},'.format(str(start)),end=''),
            requested=request_with_year_lastname_gender(year,lastname,gender,start)
            df_bm,num=bm_split_list(requested)
            if num==0: 
                break;
            save_splitdata_tocsv(df_bm,year)
            totalnum+=num
        print('total runner:{}'.format(totalnum))
                

In [35]:
get_list_year_lastname(2014,'ab')

lastname:ab,gender:11,26,51,total runner:32
lastname:ab,gender:21,26,total runner:16


In [36]:
def get_all_year(year):
    for c1 in string.ascii_lowercase:
        for c2 in string.ascii_lowercase:
            lastname=c1+c2
            get_list_year_lastname(year,lastname)

In [33]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 0C52-665E

 Directory of C:\Users\lic\Documents\PythonScripts\BostonMarathon

03/02/2017  02:23 PM    <DIR>          .
03/02/2017  02:23 PM    <DIR>          ..
03/02/2017  10:07 AM    <DIR>          .ipynb_checkpoints
03/02/2017  02:23 PM             8,262 BMwithsplits.ipynb
03/02/2017  10:06 AM            33,017 BostonMarathon.ipynb
03/02/2017  09:59 AM    <DIR>          Data
03/02/2017  09:59 AM                63 README.md
               3 File(s)         41,342 bytes
               4 Dir(s)  28,703,178,752 bytes free


In [37]:
get_all_year(2014)

lastname:aa,gender:11,26,total runner:1
lastname:aa,gender:21,26,total runner:2
lastname:ab,gender:11,26,51,total runner:32
lastname:ab,gender:21,26,total runner:16
lastname:ac,gender:11,26,total runner:15
lastname:ac,gender:21,26,total runner:14
lastname:ad,gender:11,26,51,total runner:36
lastname:ad,gender:21,26,51,total runner:33
lastname:ae,gender:11,total runner:0
lastname:ae,gender:21,26,total runner:1
lastname:af,gender:11,26,total runner:3
lastname:af,gender:21,26,total runner:2
lastname:ag,gender:11,26,total runner:15
lastname:ag,gender:21,26,total runner:16
lastname:ah,gender:11,26,total runner:14
lastname:ah,gender:21,26,total runner:9
lastname:ai,gender:11,26,total runner:8
lastname:ai,gender:21,26,total runner:8
lastname:aj,gender:11,total runner:0
lastname:aj,gender:21,26,total runner:1
lastname:ak,gender:11,26,total runner:7
lastname:ak,gender:21,26,total runner:12
lastname:al,gender:11,26,51,76,101,126,total runner:115
lastname:al,gender:21,26,51,76,101,total runner:90


ChunkedEncodingError: ('Connection broken: IncompleteRead(22 bytes read)', IncompleteRead(22 bytes read))

In [12]:
!git add BMwithsplits.ipynb

The file will have its original line endings in your working directory.


In [13]:
!git commit -m "add BMwithsplits.ipynb with splits data, version 1"

[master 6e4b27f] add BMwithsplits.ipynb with splits data, version 1
 1 file changed, 241 insertions(+)
 create mode 100644 BMwithsplits.ipynb


In [14]:
!git push

To https://github.com/flashrbt/BostonMarathon.git
   053b320..6e4b27f  master -> master
